In [5]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [5]:
import numpy as np
import gym
from gym import spaces

def winprob(a):
    return 1/(1+np.exp(-3+3*a))

def stateTransition(s, win, lead, distance): 
    if win:
        return lead + 2 * distance
    return max(s - 1, 0)

class ShippingEnv(gym.Env):
    def __init__(self):
        super(ShippingEnv, self).__init__()

        self.lead = 15
        self.distance = 20
        self.w = 100
        self.T = 1000
        self.Arrivalprob = 0.3
        self.actions = np.linspace(0.5, 2, 20)

        self.observation_space = spaces.Discrete(self.lead + 2 * self.distance + 1)
        self.action_space = spaces.Discrete(len(self.actions))

        self.state = self.observation_space.sample()
        self.time = 0

    def step(self, action):
        assert self.action_space.contains(action)

        self.time += 1

        offer_received = np.random.rand() < self.Arrivalprob
        win = np.random.rand() < winprob(self.actions[action])

        # If an offer is received, and the agent accepts it (action > 0), calculate reward and update state
        if offer_received and action > 0:
            self.state = stateTransition(self.state, win, self.lead, self.distance)
            reward = win * self.actions[action] * self.w
        else:
            self.state = max(self.state - 1, 0)
            reward = 0

        done = self.time == self.T

        return np.array(self.state).reshape((1, 1)), reward, done, {}

    def reset(self):
        self.state = self.observation_space.sample()
        self.time = 0
        return np.array(self.state).reshape((1, 1))